# CONFIGURAÇÃO

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import date

In [ ]:
data_atual = date.today().strftime('%d-%m-%Y')
FILENAME = f"{data_atual}-autismo-"

In [ ]:
conf = SparkConf()
jars = [
    'org.apache.hadoop:hadoop-aws:3.2.2', 'com.microsoft.azure:spark-mssql-connector_2.12:1.2.0', 
    'com.microsoft.sqlserver:mssql-jdbc:7.0.0.jre8',
    'com.databricks:spark-xml_2.12:0.14.0'
]
conf.set('spark.jars.packages', ','.join(jars))
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

In [ ]:
spark = SparkSession.builder\
    .appName('load-azure')\
    .config(conf=conf)\
    .getOrCreate()

# LEITURA DE ARQUIVOS

In [ ]:
person_data = spark.read.csv(f's3a://sprint-3-luiza-grupo-09-bucket-brutos/{FILENAME}-csv.csv', header=True)
person_data.show(truncate=False, vertical=True)

In [ ]:
make_data = spark.read.json(F's3a://sprint-3-luiza-grupo-09-bucket-brutos/{FILENAME}json.json')
make_data.show(truncate=False, vertical=True)

In [ ]:
region_data =  spark.read.format("com.databricks.spark.xml")\
    .option("rootTag", "data") \
    .option("rowTag", "row") \
    .load(f"s3a://sprint-3-luiza-grupo-09-bucket-brutos/{FILENAME}-xml.xml")\

region_data.show(truncate=False, vertical=True)

# JOIN DE TABELAS

In [ ]:
data_referencing = (
    person_data
        .join(make_data, person_data.id_paciente == make_data.id_renda_paciente)\
        .join(region_data, person_data.id_paciente == region_data.id_regiao_paciente)\
        .select(
            person_data.id_paciente,person_data.idade_atual,person_data.idade_descoberta,person_data.genero,
            person_data.grau,person_data.sensibilidade_sentidos,person_data.agressivo,person_data.hiperativo,
            person_data.movimentos_repetitivos,person_data.baixa_concentracao,person_data.hiperfoco,person_data.necessidade_rotina,
            person_data.dificuldade_imaginacao,person_data.introvertido,person_data.tipo_autismo,person_data.tipo_autismo_descricao,
            make_data.classe,make_data.classe_descricao,make_data.empregado,make_data.per_capita,make_data.quantidade_familia,
            make_data.total_salario,region_data.PIB_nacional,region_data.PIB_percent_BR,region_data.PIB_regional,
            region_data.area_regiao,region_data.densidade_demografica_regiao,region_data.gentilico_regiao,region_data.id_regiao_paciente,
            region_data.pais,region_data.populacao_regiao,region_data.raca_predominante_regiao,region_data.regiao
        )
)

In [ ]:
data_referencing.printSchema()

In [ ]:
data_referencing.show(truncate=False, vertical=True).to_pandas()

# SALVANDO DADOS NO BUCKET-PROCESSADOS

In [ ]:
data_referencing.write.csv(f's3a://sprint-3-luiza-grupo-09-bucket-processados/{FILENAME}_dados_agrupados.csv', header=True)

# CONEXÃO COM AZURE

In [ ]:
server = "teste-sprint3.database.windows.net"
port = 1433
db = "teste1000"
username = "urubu100"
password = "Urubu1@@"

url = f'jdbc:sqlserver://{server}:1433;database={db};'
print(url)

In [ ]:
person_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'paciente') \
    .option("user", username) \
    .option("password", password) \
    .save()

In [ ]:
make_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'renda') \
    .option("user", username) \
    .option("password", password) \
    .save()

In [ ]:
region_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'localidade') \
    .option("user", username) \
    .option("password", password) \
    .save()